<img src="https://iteso.mx/documents/27014/202031/Logo-ITESO-MinimoH.png"
     align="right"
     width="300"/>

# Modelos SARIMA

## *Modelos no lineales para pronósitico*  
- Michelle Gomez Lopez
- Juan Pablo Colome
- Carlos Moreno Labrador

---

## **<font color= #0077b6> Objetivos de la Práctica </font>**
1.  Utilizar datos de la API oficial de la MLB.
2.  Analizar la estacionariedad de la serie (Prueba de Dickey-Fuller Aumentada).
3.  Implementar la metodología Box-Jenkins para la selección del orden $(p,d,q)(P,D,Q,s)$.
4.  Utilizar `statsmodels` para realizar un modelo SARIMA.
5.  Generar pronósticos y visualizarlos interactivamente con `plotly`.

In [1]:
# @title Instalación de Librerías y Configuración

# pip install MLB-StatsAPI pmdarima plotly --quiet

import statsapi
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.stattools import adfuller, acf, pacf



## **<font color= #0077b6> Descripción de los datos </font>**

- La API de la MLB consta de múltiples estádisticas y datos de cada partido, usualmente este tipo de información se utiliza para predecir estadísticas de próximos juegos de la temporada.

- Este dataset se enfoca a recopilar la cantidad de runs por partido (sumando ambos equipos) que se juegan en la temporada. La Temporada (Season) es un maratón diario de Abril a Octubre. Cada equipo juega 162 partidos.

- Debido a que suele presentar repeticiones en patrones semanales, se utilizará un modelo SARIMA.

**Recordando:**

La Forma Canónica del modelo multiplicativo $SARIMA(p,d,q)(P,D,Q)_s$ se expresa utilizando el operador de rezago $B$:$$\phi_p(B)\Phi_P(B^s)(1-B)^d(1-B^s)^D Y_t = \theta_q(B)\Theta_Q(B^s)\epsilon_t$$

Donde:

$Y_t$: La serie de tiempo observada

$B$: Operador de rezago $B Y_t = Y_{t-1}

$\phi(B)$ y $\theta(B)$: Polinomios de la parte no estacional (AR y MA).

$\Phi(B^s)$ y $\Theta(B^s)$: Polinomios de la parte estacional (Seasonal AR y Seasonal MA).

$\epsilon_t$: Ruido blanco (Error aleatorio con media 0 y varianza constante $\sigma^2$).

In [2]:
player_id = 592450  # Aaron Judge
team_id = 147       # Yankees

def get_year_data(year):
    
    schedule = statsapi.schedule(
        start_date=f"03/01/{year}",
        end_date=f"11/30/{year}",
        team=team_id
    )
    
    data = []
    
    for game in schedule:
        
        gamePk = game["game_id"]
        game_date = game["game_date"]
        
        # Filtrar solo temporada regular
        if game.get("game_type") != "R":
            continue
        
        box = statsapi.boxscore_data(gamePk)
        
        players = {}
        players.update(box["home"]["players"])
        players.update(box["away"]["players"])
        
        for p in players.values():
            if p["person"]["id"] == player_id:
                
                hr = int(p["stats"]["batting"].get("homeRuns", 0))
                
                data.append({
                    "date": game_date,
                    "hr": hr
                })
    
    return pd.DataFrame(data)

In [3]:
all_data = []

for year in range(2017, 2025):
    print("Descargando", year)
    df_year = get_year_data(year)
    all_data.append(df_year)

df = pd.concat(all_data)



Descargando 2017
Descargando 2018
Descargando 2019
Descargando 2020
Descargando 2021
Descargando 2022
Descargando 2023
Descargando 2024


In [16]:
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")

# Filtrar solo abril (4) a septiembre (9)
df = df[df.index.month.isin([4,5,6,7,8,9])]


KeyError: 'date'

In [30]:
monthly = (
    df
    .groupby([df.index.year, df.index.month])
    .agg(
        hr_total=("hr", "sum"),
        games_played=("hr", "count")
    )
)

monthly["hr_promedio_por_juego"] = (
    monthly["hr_total"] / monthly["games_played"]
)

# Renombramos columnas de índice
monthly.index.names = ["year", "month"]

# Creamos una fecha artificial con año y mes reales
monthly["date"] = pd.to_datetime(
    monthly.index.get_level_values("year").astype(str) + "-" +
    monthly.index.get_level_values("month").astype(str) + "-01"
)

# Ahora usamos esa fecha como índice
monthly = monthly.set_index("date")

series = monthly["hr_promedio_por_juego"]



In [31]:
len(series)

44

In [32]:
print(df.index.month.value_counts().sort_index())


date
4    172
5    168
6    148
7    155
8    188
9    192
Name: count, dtype: int64


In [33]:
monthly

,hr_total,games_played,hr_promedio_por_juego
date,,,
2017-04-01,10,24,0.416667
2017-05-01,9,29,0.310345
2017-06-01,10,28,0.357143
2017-07-01,7,26,0.269231
2017-08-01,3,30,0.100000
2017-09-01,16,29,0.551724
2018-04-01,7,29,0.241379
2018-05-01,8,27,0.296296
2018-06-01,6,28,0.214286


In [37]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=series.index,
    y=series,
    mode='lines'
))

fig.update_layout(
    title='Promedio mensual de HomeRuns de Aaron Judge en la MLB',
    xaxis_title='Fecha',
    yaxis_title='Promedio de HomeRuns por mes'
)

fig.show()


In [38]:
# @title Realizamos pruebas de estacionareidad

def check_stationarity(series, title="Serie Original"):
    result = adfuller(series.dropna())
    print(f'ADF Test: {title}')
    print(f'Estadístico ADF: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    is_stationary = result[1] < 0.05
    print(f"¿Es estacionaria? {'SÍ' if is_stationary else 'NO'}\n")
    return is_stationary

# 1. Revisamos la serie original
check_stationarity(monthly["hr_promedio_por_juego"], "Nivel Original")

# 2. Aplicamos Primera Diferencia (d=1)
monthly_diff = monthly["hr_promedio_por_juego"].diff()

# 3. Revisamos la serie diferenciada
check_stationarity(monthly_diff, "Primera Diferencia (d=1)")

# Creamos una figura con 2 columnas (Subplots)
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Serie Original (No Estacionaria)", "Serie Diferenciada (Estacionaria d=1)")
)

# Gráfico 1: Serie Original
fig.add_trace(
    go.Scatter(x=monthly.index, y=monthly["hr_promedio_por_juego"], name='Original'),
    row=1, col=1
)

# Gráfico 2: Serie Diferenciada
fig.add_trace(
    go.Scatter(x=monthly_diff.index, y=monthly_diff, name='Diferenciada'),
    row=1, col=2
)

# Ajustes de diseño
fig.update_layout(
    title_text="Comparativa: Efecto de la Diferenciación",
    showlegend=False, # Ocultamos leyenda
    height=500
)

fig.show()

ADF Test: Nivel Original
Estadístico ADF: -6.0675
p-value: 0.0000
¿Es estacionaria? SÍ

ADF Test: Primera Diferencia (d=1)
Estadístico ADF: -7.4020
p-value: 0.0000
¿Es estacionaria? SÍ



In [42]:

# @title Graficamos ACF y PACF

ts_analysis = monthly.dropna()

# Parámetros
lags = 7  # 30 días
alpha = 0.05 # Nivel de significancia del 95%

# Cálculo de valores ACF y PACF
acf_vals = acf(ts_analysis['hr_promedio_por_juego'], nlags=lags, alpha=alpha)[0][1:]
pacf_vals = pacf(ts_analysis['hr_promedio_por_juego'], nlags=lags, alpha=alpha)[0][1:]

# Colocamos manualmente el intervalo de confianza para plotly
n = len(ts_analysis)
conf_interval = 1.96 / np.sqrt(n)


fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("Función de Autocorrelación (ACF) - Determina MA(q)",
                                    "Autocorrelación Parcial (PACF) - Determina AR(p)"),
                    vertical_spacing=0.15)

# ACF

fig.add_trace(go.Bar(
    x=list(range(1, lags+1)), y=acf_vals,
    name='ACF', marker_color='rgb(31, 119, 180)', showlegend=False
), row=1, col=1)

# Intervalos de confianza (Sombreado)
fig.add_shape(type="rect",
    x0=0.5, y0=-conf_interval, x1=lags+0.5, y1=conf_interval,
    line=dict(color="rgba(0,0,0,0)"), fillcolor="rgba(0,0,0,0.1)",
    row=1, col=1
)
# Líneas límite
fig.add_hline(y=conf_interval, line_dash="dash", line_color="gray", row=1, col=1)
fig.add_hline(y=-conf_interval, line_dash="dash", line_color="gray", row=1, col=1)

# PACF

fig.add_trace(go.Bar(
    x=list(range(1, lags+1)), y=pacf_vals,
    name='PACF', marker_color='rgb(255, 127, 14)', showlegend=False
), row=2, col=1)

# Intervalos de confianza
fig.add_shape(type="rect",
    x0=0.5, y0=-conf_interval, x1=lags+0.5, y1=conf_interval,
    line=dict(color="rgba(0,0,0,0)"), fillcolor="rgba(0,0,0,0.1)",
    row=2, col=1
)
# Líneas límite
fig.add_hline(y=conf_interval, line_dash="dash", line_color="gray", row=2, col=1)
fig.add_hline(y=-conf_interval, line_dash="dash", line_color="gray", row=2, col=1)


fig.update_layout(
    title='<b>Diagnóstico de Estructura: ACF y PACF</b><br><sup>Serie Diferenciada</sup>',
    template='plotly_white',
    height=700,
    bargap=0.8 # Barras delgadas estilo lollipop
)

# Resaltar lags estacionales (7, 14, 21, 28) con líneas verticales rojas tenues
for i in [6, 12, 18, 24]:
    fig.add_vline(x=i, line_width=1, line_dash="dot", line_color="red", opacity=0.5)

fig.show()

In [43]:
# @title Realizamos modelo y graficamos
TEST_DAYS = 6  # Últimos 6 meses para test (abril a septiembre)

train = monthly["hr_promedio_por_juego"].iloc[:-TEST_DAYS]
test = monthly["hr_promedio_por_juego"].iloc[-TEST_DAYS:]

# Con utilizar una diferenciacion es suficiente, tu decides cual utilizar
# (revisa la forma canónica) para verificar que el modelo despues de simplificar
# ya se trata de una serie estacionaria
model = SARIMAX(train,
                order=(1, 0, 1),
                seasonal_order=(1, 0, 0, 6))

results = model.fit(disp=False)

# Predecimos n pasos hacia el futuro (donde n = tamaño del test)
forecast_object = results.get_forecast(steps=len(test))
forecast_vals = forecast_object.predicted_mean
conf_int = forecast_object.conf_int(alpha=0.05) # Intervalo del 95%

# Metricas de error
rmse = np.sqrt(mean_squared_error(test, forecast_vals))
# Ojo con el MAPE, hay algunos días con 0 carreras y esto da valores muy grandes
mape = mean_absolute_percentage_error(test, forecast_vals)
mae = mean_absolute_error(test, forecast_vals)


print(f"\n--- Errores del modelo ---")
print(f"RMSE: {rmse:.2f} Carreras")
print(f"MAPE: {mape:.2%}")
print(f"MAE: {mae:.2f} Carreras")

print(results.summary())

# Grafica
fig = go.Figure()

# Train
fig.add_trace(go.Scatter(
    x=train.index, y=train,
    mode='lines',
    name='Train',
    line=dict(color='rgba(100, 100, 100, 0.6)', width=1.5)
))

# Test
fig.add_trace(go.Scatter(
    x=test.index, y=test,
    name='Test',
    line=dict(color='#1f77b4', width=3),
    marker=dict(size=6)
))

# Forecast
fig.add_trace(go.Scatter(
    x=test.index, y=forecast_vals,
    name='SARIMA',
    line=dict(color='#ff7f0e', width=3, dash='dot')
))

# Intervalos de Confianza
fig.add_trace(go.Scatter(
    x=conf_int.index, y=conf_int.iloc[:, 0],
    mode='lines', line=dict(width=0), showlegend=False, hoverinfo='skip'
))
fig.add_trace(go.Scatter(
    x=conf_int.index, y=conf_int.iloc[:, 1],
    mode='lines', line=dict(width=0), fill='tonexty',
    fillcolor='rgba(255, 127, 14, 0.2)',
    name='Int. Confianza 95%', hoverinfo='skip'
))

# Titulos
fig.update_layout(
    title=f'<b>Modelo SARIMA: Pronóstico de Carreras MLB</b><br><sup>',
    xaxis_title='Fecha',
    yaxis_title='Total de Carreras',
    legend=dict(x=0, y=1, bgcolor='rgba(255,255,255,0.8)'),
    hovermode="x unified"
)

fig.show()


--- Errores del modelo ---
RMSE: 0.11 Carreras
MAPE: 27.64%
MAE: 0.09 Carreras
                                     SARIMAX Results                                      
Dep. Variable:              hr_promedio_por_juego   No. Observations:                   38
Model:             SARIMAX(1, 0, 1)x(1, 0, [], 6)   Log Likelihood                  21.365
Date:                            Wed, 18 Feb 2026   AIC                            -34.731
Time:                                    19:25:59   BIC                            -28.180
Sample:                                         0   HQIC                           -32.400
                                             - 38                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9980      0.006  

c:\Users\colom\OneDrive - ITESO\iteso\8vo semestre\modelos_no_lineales\mlb_time_serie\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\colom\OneDrive - ITESO\iteso\8vo semestre\modelos_no_lineales\mlb_time_serie\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\colom\OneDrive - ITESO\iteso\8vo semestre\modelos_no_lineales\mlb_time_serie\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\colom\OneDrive - ITESO\iteso\8vo semestre\modelos_no_lineales\mlb_time_serie\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: 